In [27]:
import gradio as gr
from datetime import datetime
import pandas as pd
import plotly.express as px

def calculate_bmi_units(name, age, height, weight, unit):
    try:
        if unit == "Imperial (in/lb)":
            height_m = height * 0.0254
            weight_kg = weight * 0.453592
        else:
            height_m = height / 100
            weight_kg = weight

        bmi = round(weight_kg / (height_m ** 2), 2)
        if bmi < 18.5:
            category = "Underweight"
        elif 18.5 <= bmi < 25:
            category = "Normal weight"
        elif 25 <= bmi < 30:
            category = "Overweight"
        else:
            category = "Obese"

        timestamp = datetime.now().isoformat()

        conn = sqlite3.connect(DB_FILE)
        cursor = conn.cursor()
        cursor.execute("INSERT INTO bmi_records (name, age, height_cm, weight_kg, bmi, category, timestamp) VALUES (?, ?, ?, ?, ?, ?, ?)",
                       (name, age, round(height_m*100,2), round(weight_kg,2), bmi, category, timestamp))
        conn.commit()
        conn.close()

        conn = sqlite3.connect(DB_FILE)
        df = pd.read_sql_query("SELECT * FROM bmi_records WHERE name=?", conn, params=(name,))
        conn.close()
        if not df.empty:
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            fig = px.line(df, x='timestamp', y='bmi', title=f"{name}'s BMI Over Time", markers=True)
        else:
            fig = None

        return f"{name}, your BMI is {bmi} ({category})", fig
    except Exception as e:
        return f"Error: {str(e)}", None

def log_food(name, food_item, calories):
    try:
        timestamp = datetime.now().isoformat()
        conn = sqlite3.connect(DB_FILE)
        cursor = conn.cursor()
        cursor.execute("INSERT INTO food_log (name, food_item, calories, timestamp) VALUES (?, ?, ?, ?)",
                       (name, food_item, calories, timestamp))
        conn.commit()
        conn.close()

        conn = sqlite3.connect(DB_FILE)
        df = pd.read_sql_query("SELECT * FROM food_log WHERE name=?", conn, params=(name,))
        conn.close()
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        today = datetime.now().date()
        daily_total = df[df['timestamp'].dt.date == today]['calories'].sum()

        fig = px.bar(df, x='timestamp', y='calories', title=f"{name}'s Food Intake", labels={'calories':'Calories'})

        return f"{food_item} ({calories} kcal) logged for {name}. Today's total: {daily_total} kcal", fig
    except Exception as e:
        return f"Error: {str(e)}", None

def calculate_bmr_tdee(name, age, gender, height_cm, weight_kg, activity_level):
    try:
        if gender.lower() == "male":
            bmr = 88.362 + (13.397 * weight_kg) + (4.799 * height_cm) - (5.677 * age)
        else:
            bmr = 447.593 + (9.247 * weight_kg) + (3.098 * height_cm) - (4.330 * age)

        activity_factors = {
            "Sedentary": 1.2,
            "Lightly active": 1.375,
            "Moderately active": 1.55,
            "Very active": 1.725,
            "Extra active": 1.9
        }
        tdee = bmr * activity_factors.get(activity_level, 1.2)

        timestamp = datetime.now().isoformat()
        conn = sqlite3.connect(DB_FILE)
        cursor = conn.cursor()
        cursor.execute("""INSERT INTO bmr_tdee_log
                          (name, age, gender, height_cm, weight_kg, activity_level, bmr, tdee, timestamp)
                          VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)""",
                       (name, age, gender, height_cm, weight_kg, activity_level, round(bmr,2), round(tdee,2), timestamp))
        conn.commit()
        conn.close()

        return (f"{name}'s BMR: {bmr:.1f} kcal\n"
                f"TDEE (maintenance): {tdee:.1f} kcal\n"
                f"Calories for weight loss: {tdee - 500:.1f} kcal\n"
                f"Calories for weight gain: {tdee + 500:.1f} kcal")
    except Exception as e:
        return f"Error: {str(e)}"

def view_logs_all():
    try:
        conn = sqlite3.connect(DB_FILE)
        bmi_df = pd.read_sql_query("SELECT * FROM bmi_records", conn)
        food_df = pd.read_sql_query("SELECT * FROM food_log", conn)
        bmr_df = pd.read_sql_query("SELECT * FROM bmr_tdee_log", conn)
        conn.close()

        result = "---- All BMI Logs ----\n"
        result += bmi_df.to_string(index=False) if not bmi_df.empty else "No BMI records found."
        result += "\n\n---- All Food Logs ----\n"
        result += food_df.to_string(index=False) if not food_df.empty else "No food records found."
        result += "\n\n---- All BMR/TDEE Logs ----\n"
        result += bmr_df.to_string(index=False) if not bmr_df.empty else "No BMR/TDEE records found."

        return result
    except Exception as e:
        return f"Error reading logs: {str(e)}"

with gr.Blocks(theme=gr.themes.Default(primary_hue="pink", secondary_hue="blue")) as iface:
    gr.Markdown("# 🧮 BMI & 🍎 Food Tracker (SQLite Version)")

    with gr.Tab("Calculate BMI"):
        name = gr.Textbox(label="Name")
        age = gr.Number(label="Age", precision=0)
        unit = gr.Radio(choices=["Metric (cm/kg)", "Imperial (in/lb)"], label="Units", value="Metric (cm/kg)")
        height = gr.Number(label="Height")
        weight = gr.Number(label="Weight")
        bmi_button = gr.Button("Calculate BMI")
        bmi_output = gr.Textbox(label="BMI Result")
        bmi_chart = gr.Plot()
        bmi_button.click(fn=calculate_bmi_units,
                         inputs=[name, age, height, weight, unit],
                         outputs=[bmi_output, bmi_chart])

    with gr.Tab("Daily Metabolic Rate"):
        name2 = gr.Textbox(label="Name")
        age2 = gr.Number(label="Age")
        gender = gr.Radio(choices=["Male", "Female"], label="Gender")
        height2 = gr.Number(label="Height (cm)")
        weight2 = gr.Number(label="Weight (kg)")
        activity = gr.Dropdown(choices=["Sedentary", "Lightly active", "Moderately active", "Very active", "Extra active"], label="Activity Level")
        tdee_button = gr.Button("Calculate BMR & TDEE")
        tdee_output = gr.Textbox(label="Results")
        tdee_button.click(fn=calculate_bmr_tdee,
                          inputs=[name2, age2, gender, height2, weight2, activity],
                          outputs=tdee_output)

    with gr.Tab("Food Tracker"):
        name3 = gr.Textbox(label="Name")
        food = gr.Textbox(label="Food Item")
        calories = gr.Number(label="Calories")
        log_button = gr.Button("Log Food")
        log_output = gr.Textbox(label="Log Result")
        food_chart = gr.Plot()
        log_button.click(fn=log_food,
                         inputs=[name3, food, calories],
                         outputs=[log_output, food_chart])

    with gr.Tab("View Logs"):
        view_button = gr.Button("View All Logs")
        logs_output = gr.Textbox(lines=25, label="All Logs")
        view_button.click(fn=view_logs_all,
                          inputs=None,
                          outputs=logs_output)

iface.launch(share=True)

import sqlite3

DB_FILE = "BMI_Calculator.db"

def init_db():
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS bmi_records (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        age INTEGER,
        height_cm REAL,
        weight_kg REAL,
        bmi REAL,
        category TEXT,
        timestamp TEXT
    )
    """)

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS food_log (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        food_item TEXT,
        calories REAL,
        timestamp TEXT
    )
    """)

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS bmr_tdee_log (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        age INTEGER,
        gender TEXT,
        height_cm REAL,
        weight_kg REAL,
        activity_level TEXT,
        bmr REAL,
        tdee REAL,
        timestamp TEXT
    )
    """)

    conn.commit()
    conn.close()

init_db()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://30ef220c0ba042b6c4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
